In [47]:
import torch
import torch.nn as nn
from transformers import BertModel, BertTokenizer, AutoTokenizer


class BertHidden(nn.Module):
    def __init__(self):
        super(BertHidden, self).__init__()
        
        # initialize the BERT model and tokenizer
        self.bert_model = BertModel.from_pretrained('bert-base-cased')
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
        self.autotokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
        
        # add a linear layer to predict the output class
        # self.linear = nn.Linear(self.bert_model.config.hidden_size, num_classes)
        
    def forward(self, inputs):
        # tokenize each word in the input
        res= []

        for input in inputs:
            tokenized_inputs = [self.tokenizer.encode(inp, add_special_tokens=True) for inp in input]
            autotokenized = self.autotokenizer(input, padding='longest', truncation=True)['input_ids']
            print(autotokenized)
            
            # # pad the tokenized inputs to the same length
            max_len = max([len(inp) for inp in tokenized_inputs])
            padded_inputs = torch.zeros(len(input), max_len).long()
            for i, inp in enumerate(tokenized_inputs):
                padded_inputs[i, :len(inp)] = torch.tensor(inp)
            
            # pass the padded inputs through the BERT model
            outputs = self.bert_model(padded_inputs)
            
            # get the output of the [CLS] token
            pooled_output = outputs[1]
            res.append(pooled_output)

        return res

        # pass the pooled output through the linear layer to get the predicted class
        logits = self.linear(pooled_output)
        
        return logits

In [48]:
bert = BertHidden()

# generate a list of input strings
inputs = [["This is the presentence input","This is the claim input", "This is the postsentence input"],["This is the zero input","This is the claim input", "This is the second input"]]
labels = [0,1,0]

# pass the inputs through the BertTokenized model
hidden_layer = bert(inputs)

print(hidden_layer[0])

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[[101, 1188, 1110, 1103, 1675, 7008, 7758, 102, 0], [101, 1188, 1110, 1103, 3548, 7758, 102, 0, 0], [101, 1188, 1110, 1103, 8345, 23771, 3633, 7758, 102]]
[[101, 1188, 1110, 1103, 6756, 7758, 102], [101, 1188, 1110, 1103, 3548, 7758, 102], [101, 1188, 1110, 1103, 1248, 7758, 102]]
tensor([[-0.6550,  0.4765,  0.9999,  ...,  1.0000, -0.6728,  0.9948],
        [-0.6672,  0.4955,  0.9999,  ...,  1.0000, -0.6377,  0.9893],
        [-0.6837,  0.4373,  0.9999,  ...,  1.0000, -0.7870,  0.9848]],
       grad_fn=<TanhBackward0>)
